In [1]:
import numpy as np
import sys 
import os
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.linalg
from matplotlib import cm
import sympy
from sympy import *
import random
import numpy.polynomial.polynomial as poly
import matplotlib.ticker as tkr

In [3]:
def extract_freq(file):
    """ 
    This function extract vibrational frequencies and vibrational coordinates from a log file. Returns a list of vibrational
    frequencies, freq_list, and a list of reduced masses, red_list.
    """
    freq_list=[] #A list of vibrational frequencies.
    red_list= [] #A list of reduced masses of the vibrational modes.
    
    c=2.998*10**(10) #speed of light in cm/s.
    amu_kg= 1.66053904*10**-27 #amu to kg conversion factor.
   
    with open(file, 'r') as file:
        line = file.readline()
        while line:

            if 'Frequencies' in line :
                freq1=0
                freq2=0
                freq3=0
                red1 = 0
                red2 = 0
                red3 = 0
                
                data = line.split()
                freq1 = np.float(data[2])
                freq_list.append(freq1)
                freq2 = np.float(data[3])
                freq_list.append(freq2)
                freq3 = np.float(data[4])
                freq_list.append(freq3)
                line= file.readline()
                
                data2 = line.split()
                red1 = np.float(data2[3])
                red_list.append(red1)
                red2 = np.float(data2[4])
                red_list.append(red2)
                red3 = np.float(data2[5])
                red_list.append(red3)
            line = file.readline()
            
    freq_list=np.array(freq_list)
    freq_list*=c
    red_list=np.array(red_list)
    red_list*=amu_kg
    
    
            
    return freq_list, red_list

In [4]:
"""
Extract vibrational indices and x, y and z components of the transition dipole moment
from 5 txt-files generated from the output files by the 
Extract_sign.py script
"""

steps=19
vib1_mat=np.zeros((steps,steps))
vib2_mat=np.zeros((steps,steps))
tdm_x_mat=np.zeros((steps,steps))
tdm_y_mat=np.zeros((steps,steps))
tdm_z_mat=np.zeros((steps,steps))
i_list=[]
j_list=[]
k_list=[]
l_list=[]

file1=open('vib1_data.txt', 'r') #text file with the vibrational index 1
line = file1.readline()
i_list = line.split()
for i in range(len(i_list)):
    i_list[i]=float(i_list[i])
    k=int(i_list[i])-9
    k_list.append(k)
    
    
file2=open('vib2_data.txt', 'r') #text file with the vibrational index 2
line = file2.readline()
j_list = line.split()
for i in range(len(j_list)):
    j_list[i]=float(j_list[i])
    l=int(j_list[i])-9
    l_list.append(l)
for i in range(len(i_list)):   
   
    vib1_mat[int(i_list[i]),int(j_list[i])]=k_list[i]
    vib2_mat[int(i_list[i]),int(j_list[i])]=l_list[i]

file3=open('tdm_x_data.txt', 'r') #text file with the x components of the transition dipole moments
line = file3.readline()
tdm_x_list = line.split()
for i in range(len(tdm_x_list)):
    tdm_x_list[i]=float(tdm_x_list[i])/1.889725989
for i in range(len(i_list)):
    tdm_x_mat[int(i_list[i]),int(j_list[i])]=tdm_x_list[i]
    
file4=open('tdm_y_data.txt', 'r') #text file with the y components of the transition dipole moments
line = file4.readline()
tdm_y_list = line.split()
for i in range(len(tdm_y_list)):
    tdm_y_list[i]=float(tdm_y_list[i])/1.889725989
for i in range(len(i_list)):
    tdm_y_mat[int(i_list[i]),int(j_list[i])]=tdm_y_list[i]    

file5=open('tdm_z_data.txt', 'r') #text file with the z components of the transition dipole moments
line = file5.readline()
tdm_z_list = line.split()
for i in range(len(tdm_z_list)):
    tdm_z_list[i]=float(tdm_z_list[i])/1.889725989
for i in range(len(i_list)):
    tdm_z_mat[int(i_list[i]),int(j_list[i])]=tdm_z_list[i] 


FileNotFoundError: [Errno 2] No such file or directory: 'vib1_data.txt'

In [ ]:
def oscillator(file):
    """
    This function extracts the wavelength of the transition from a log. file
    """
    h=6.62608*10**(-34) #Planck's constant /J*s
    me=9.10938*10**(-31) #Electron mass/ kg
    c=2.998*10**(17) #Speed of light/ nm/s
    wl=0
    with open(file, 'r') as file:
        line = file.readline()
        while line:
            if 'Excited State   1' in line :
                data = line.split()
                wl = np.float(data[6])
            line = file.readline()
    
    ex_freq=c/wl
    
    return ex_freq

In [ ]:
def fit(T,order,S):
    """
    This function fits a dataset consisting of two vibrational coordinates and corresponding
    transition dipole moments to a polynomial of orders ranging from 1 to 12 subtracting
    a number of unique datapoints, S. S indicates the size of the grid that should be fitted. 
    """

    if T==1:
        Z=tdm_x_mat.flatten()
    elif T==2:
        Z=tdm_y_mat.flatten()
    elif T==3:
        Z=tdm_z_mat.flatten()
        
        X=vib1_mat.flatten()
        Y=vib2_mat.flatten()
    
    if not S==0:
        #Generates a list of S random indices 
        a = set()
        sampleSize = S
        answerSize = 0
        while answerSize < sampleSize:
            r = random.randint(0,len(X)-1)
            if r not in a:
                answerSize += 1
                a.add(r)
        a=np.array(list(a))
   
        Y=np.delete(Y,a)
        X=np.delete(X,a)
        Z=np.delete(Z,a)
    data = np.c_[X,Y,Z]

    # regular grid covering the domain of the data
    X,Y = np.meshgrid(np.linspace(-9,9,18), np.linspace(-9,9,18))
   
    if order == 1:
        A = np.c_[data[:,0], data[:,1], np.ones(data.shape[0])]
        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])    # coefficients
        # evaluate it on grid
        Z = C[0]*X + C[1]*Y + C[2]

    elif order == 2:
        A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1),\
                  data[:,:2]**2]
        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])
        # evaluate it on a grid
        Z  = C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0]

    elif order == 3:
        A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1),
        data[:,:2]**2, data[:,0]*(data[:,1]**2),
        data[:,1]*(data[:,0]**2),data[:,:2]**3]
        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])
        # evaluate it on a grid
        Z  = C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] + C[6]*X*Y*Y\
        + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3

    elif order == 4:
        A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1),\
                  data[:,:2]**2, data[:,0]*(data[:,1]**2), data[:,1]*(data[:,0]**2),\
                  data[:,:2]**3,data[:,0]*data[:,1]**3,data[:,1]*data[:,0]**3,\
                  data[:,0]**2*data[:,1]**2,data[:,:2]**4]
        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])
        # evaluate it on a grid
        Z  = C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] + C[6]*X*Y*Y\
        + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 +C[10]*X*Y**3 + C[11]*Y*X**3 + C[12]*X**2*Y**2\
        + C[13]*X**4 + C[14]*Y**4

    elif order == 5:
        A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1),\
                  data[:,:2]**2, data[:,0]*(data[:,1]**2),data[:,1]*(data[:,0]**2),\
                  data[:,:2]**3,data[:,0]*data[:,1]**3,data[:,1]*data[:,0]**3,\
                  data[:,0]**2*data[:,1]**2,data[:,:2]**4,data[:,0]*data[:,1]**4,\
                  data[:,1]*data[:,0]**4,data[:,0]**2*data[:,1]**3,data[:,1]**2*data[:,0]**3, \
                  data[:,:2]**5]
        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])
        # evaluate it on a grid
        Z  = C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] + C[6]*X*Y*Y\
        + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 +C[10]*X*Y**3 + C[11]*Y*X**3 + C[12]*X**2*Y**2\
        + C[13]*X**4 + C[14]*Y**4 + C[15]*X*Y**4 + C[16]*Y*X**4 + C[17]*X**2*Y**3\
        + C[18]*X**3*Y**2 + C[19]*X**5 + C[20]*Y**5

    elif order == 6:
        A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1), data[:,:2]**2,\
                  data[:,0]*(data[:,1]**2),data[:,1]*(data[:,0]**2),data[:,:2]**3\
                  ,data[:,0]*data[:,1]**3,data[:,1]*data[:,0]**3,data[:,0]**2*data[:,1]**2,\
                  data[:,:2]**4,data[:,0]*data[:,1]**4,data[:,1]*data[:,0]**4,data[:,0]**2*data[:,1]**3\
                  ,data[:,1]**2*data[:,0]**3,data[:,:2]**5,data[:,0]**5*data[:,1],data[:,1]**5*data[:,0],\
                  data[:,0]**4*data[:,1]**2, data[:,1]**4*data[:,0]**2,data[:,0]**3*data[:,1]**3,\
                  data[:,:2]**6]
        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])
        # evaluate it on a grid
        Z  = C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] + C[6]*X*Y*Y \
        + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 +C[10]*X*Y**3 + C[11]*Y*X**3 + C[12]*X**2*Y**2\
        + C[13]*X**4 + C[14]*Y**4 + C[15]*X*Y**4 + C[16]*Y*X**4 + C[17]*X**2*Y**3 +\
        C[18]*X**3*Y**2 + C[19]*X**5 + C[20]*Y**5 + C[21]*X**5*Y + C[22]*Y**5*X +  C[23]*X**4*Y**2\
        + C[24]*Y**4*X**2 + C[25]*X**3*Y**3 + C[26]*X**6 + C[27]*Y**6
    
    elif order == 7:
        A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1), data[:,:2]**2,\
                  data[:,0]*(data[:,1]**2), data[:,1]*(data[:,0]**2),data[:,:2]**3,data[:,0]*data[:,1]**3\
                  ,data[:,1]*data[:,0]**3,data[:,0]**2*data[:,1]**2,data[:,:2]**4,\
                  data[:,0]*data[:,1]**4,data[:,1]*data[:,0]**4,data[:,0]**2*data[:,1]**3\
                  ,data[:,1]**2*data[:,0]**3,data[:,:2]**5,data[:,0]**5*data[:,1],data[:,1]**5*data[:,0]\
                  , data[:,0]**4*data[:,1]**2, data[:,1]**4*data[:,0]**2,data[:,0]**3*data[:,1]**3\
                  ,data[:,:2]**6,data[:,0]**6*data[:,1],data[:,1]**6*data[:,0],data[:,0]**5*data[:,1]**2,\
                  data[:,1]**5*data[:,0]**2,data[:,0]**4*data[:,1]**3,data[:,1]**4*data[:,0]**3\
                  ,data[:,:2]**7]
        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])
        # evaluate it on a grid
        Z  = C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] + C[6]*X*Y*Y\
        + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 + C[10]*X*Y**3 + C[11]*Y*X**3 + C[12]*X**2*Y**2\
        + C[13]*X**4 + C[14]*Y**4 + C[15]*X*Y**4 + C[16]*Y*X**4 + C[17]*X**2*Y**3+ C[18]*X**3*Y**2 \
        + C[19]*X**5 + C[20]*Y**5 + C[21]*X**5*Y + C[22]*Y**5*X +  C[23]*X**4*Y**2 + C[24]*Y**4*X**2 +\
        C[25]*X**3*Y**3 + C[26]*X**6 + C[27]*Y**6 + C[28]*X**6*Y + C[29]*X*Y**6 + C[30]*X**5*Y**2\
        + C[31]*X**2*Y**5 +C[32]*X**4*Y**3 + C[33]*X**3*Y**4 + C[34]*X**7 + C[35]*Y**7

    elif order == 8:
        A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1), data[:,:2]**2,\
                  \
                  data[:,0]*(data[:,1]**2),data[:,1]*(data[:,0]**2),data[:,:2]**3\
                  \
                  ,data[:,0]*data[:,1]**3,data[:,1]*data[:,0]**3,data[:,0]**2*data[:,1]**2,data[:,:2]**4\
                  \
                  ,data[:,0]*data[:,1]**4,data[:,1]*data[:,0]**4,data[:,0]**2*data[:,1]**3,\
                  data[:,1]**2*data[:,0]**3,data[:,:2]**5\
                  ,data[:,0]**5*data[:,1],data[:,1]**5*data[:,0], data[:,0]**4*data[:,1]**2, \
                  data[:,1]**4*data[:,0]**2,  data[:,0]**3*data[:,1]**3,data[:,:2]**6\
                  \
                  ,data[:,0]**6*data[:,1],data[:,1]**6*data[:,0],data[:,0]**5*data[:,1]**2\
                  ,data[:,1]**5*data[:,0]**2,data[:,0]**4*data[:,1]**3,data[:,1]**4*data[:,0]**3,\
                  data[:,:2]**7
                  \
                  ,data[:,0]**7*data[:,1],data[:,1]**7*data[:,0], data[:,0]**6*data[:,1]**2\
                  , data[:,1]**6*data[:,0]**2,data[:,0]**5*data[:,1]**3, data[:,1]**5*data[:,0]**3,\
                  data[:,0]**4*data[:,1]**4, data[:,:2]**8]

        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])
        # evaluate it on a grid
        Z  = C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] \
        \
        + C[6]*X*Y*Y + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 \
        \
        + C[10]*X*Y**3 + C[11]*Y*X**3 + C[12]*X**2*Y**2 + C[13]*X**4 + C[14]*Y**4 \
        \
        + C[15]*X*Y**4 + C[16]*Y*X**4 + C[17]*X**2*Y**3 + C[18]*X**3*Y**2 + C[19]*X**5 + C[20]*Y**5 \
        \
        + C[21]*X**5*Y + C[22]*Y**5*X +  C[23]*X**4*Y**2 + C[24]*Y**4*X**2 + C[25]*X**3*Y**3 + C[26]*X**6 \
        \
        + C[27]*Y**6 + C[28]*X**6*Y + C[29]*X*Y**6 + C[30]*X**5*Y**2 + C[31]*X**2*Y**5 +C[32]*X**4*Y**3\
        + C[33]*X**3*Y**4 +C[34]*X**7 + C[35]*Y**7 \
        \
        + C[36]*X**7*Y + C[37]*X*Y**7 + C[38]*X**6*Y**2 + C[39]*Y**6*X**2 + C[40]*X**5*Y**3 +\
        C[41]*Y**5*X**3 + C[42]*X**4*Y**4 +C[43]*X**8 + C[44]*Y**8

    elif order == 9:
        A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1), data[:,:2]**2,\
                  \
                  data[:,0]*(data[:,1]**2),data[:,1]*(data[:,0]**2),data[:,:2]**3\
                  \
                  ,data[:,0]*data[:,1]**3,data[:,1]*data[:,0]**3,data[:,0]**2*data[:,1]**2,data[:,:2]**4\
                  \
                  ,data[:,0]*data[:,1]**4,data[:,1]*data[:,0]**4,data[:,0]**2*data[:,1]**3,data[:,1]**2*data[:,0]**3,data[:,:2]**5\
                  \
                  ,data[:,0]**5*data[:,1],data[:,1]**5*data[:,0], data[:,0]**4*data[:,1]**2, data[:,1]**4*data[:,0]**2,
                  data[:,0]**3*data[:,1]**3,data[:,:2]**6\
                  \
                  ,data[:,0]**6*data[:,1],data[:,1]**6*data[:,0],data[:,0]**5*data[:,1]**2,data[:,1]**5*data[:,0]**2\
                  ,data[:,0]**4*data[:,1]**3,data[:,1]**4*data[:,0]**3,data[:,:2]**7\
                  \
                  ,data[:,0]**7*data[:,1],data[:,1]**7*data[:,0], data[:,0]**6*data[:,1]**2, data[:,1]**6*data[:,0]**2,\
                  data[:,0]**5*data[:,1]**3, data[:,1]**5*data[:,0]**3, data[:,0]**4*data[:,1]**4, data[:,:2]**8,\
                 \
                 data[:,0]**8*data[:,1],data[:,1]**8*data[:,0], data[:,0]**7*data[:,1]**2, data[:,1]**7*data[:,0]**2,\
                  data[:,0]**6*data[:,1]**3, data[:,1]**6*data[:,0]**3, data[:,0]**5*data[:,1]**4, data[:,1]**5*data[:,0]**4,\
                  data[:,:2]**9]

        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])
        # evaluate it on a grid
        Z  = C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] \
        \
        + C[6]*X*Y*Y + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 \
        \
        + C[10]*X*Y**3 + C[11]*Y*X**3 + C[12]*X**2*Y**2 + C[13]*X**4 + C[14]*Y**4 \
        \
        + C[15]*X*Y**4 + C[16]*Y*X**4 + C[17]*X**2*Y**3 + C[18]*X**3*Y**2 + C[19]*X**5 + C[20]*Y**5 \
        \
        + C[21]*X**5*Y + C[22]*Y**5*X +  C[23]*X**4*Y**2 + C[24]*Y**4*X**2 + C[25]*X**3*Y**3 + C[26]*X**6 \
        \
        + C[27]*Y**6 + C[28]*X**6*Y + C[29]*X*Y**6 + C[30]*X**5*Y**2 + C[31]*X**2*Y**5 +C[32]*X**4*Y**3 + C[33]*X**3*Y**4 +\
        C[34]*X**7 + C[35]*Y**7 \
        \
        + C[36]*X**7*Y + C[37]*X*Y**7 + C[38]*X**6*Y**2 + C[39]*Y**6*X**2 + C[40]*X**5*Y**3 + C[41]*Y**5*X**3 + C[42]*X**4*Y**4 +\
        C[43]*X**8 + C[44]*Y**8\
        \
        + C[45]*X**8*Y + C[46]*X*Y**8 + C[47]*X**7*Y**2 + C[48]*Y**7*X**2 + C[49]*X**6*Y**3 + C[50]*Y**6*X**3 + C[51]*X**5*Y**4 +\
        C[52]*Y**5*X**4 + C[53]*X**9 + C[54]*Y**9

    elif order == 10:
        A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1), data[:,:2]**2,\
                  \
                  data[:,0]*(data[:,1]**2),data[:,1]*(data[:,0]**2),data[:,:2]**3\
                  \
                  ,data[:,0]*data[:,1]**3,data[:,1]*data[:,0]**3,data[:,0]**2*data[:,1]**2,data[:,:2]**4\
                  \
                  ,data[:,0]*data[:,1]**4,data[:,1]*data[:,0]**4,data[:,0]**2*data[:,1]**3,data[:,1]**2*data[:,0]**3,data[:,:2]**5\
                  \
                  ,data[:,0]**5*data[:,1],data[:,1]**5*data[:,0], data[:,0]**4*data[:,1]**2, data[:,1]**4*data[:,0]**2,
                  data[:,0]**3*data[:,1]**3,data[:,:2]**6\
                  \
                  ,data[:,0]**6*data[:,1],data[:,1]**6*data[:,0],data[:,0]**5*data[:,1]**2,data[:,1]**5*data[:,0]**2\
                  ,data[:,0]**4*data[:,1]**3,data[:,1]**4*data[:,0]**3,data[:,:2]**7\
                  \
                  ,data[:,0]**7*data[:,1],data[:,1]**7*data[:,0], data[:,0]**6*data[:,1]**2, data[:,1]**6*data[:,0]**2,\
                  data[:,0]**5*data[:,1]**3, data[:,1]**5*data[:,0]**3, data[:,0]**4*data[:,1]**4, data[:,:2]**8,\
                 \
                 data[:,0]**8*data[:,1],data[:,1]**8*data[:,0], data[:,0]**7*data[:,1]**2, data[:,1]**7*data[:,0]**2,\
                  data[:,0]**6*data[:,1]**3, data[:,1]**6*data[:,0]**3, data[:,0]**5*data[:,1]**4, data[:,1]**5*data[:,0]**4,\
                  data[:,:2]**9\
                 \
                 ,data[:,0]**9*data[:,1],data[:,1]**9*data[:,0], data[:,0]**8*data[:,1]**2, data[:,1]**8*data[:,0]**2,\
                  data[:,0]**7*data[:,1]**3, data[:,1]**7*data[:,0]**3, data[:,0]**6*data[:,1]**4, data[:,1]**6*data[:,0]**4,\
                  data[:,0]**5*data[:,1]**5,data[:,:2]**10]

        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])
        # evaluate it on a grid
        Z  = C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] \
        \
        + C[6]*X*Y*Y + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 \
        \
        + C[10]*X*Y**3 + C[11]*Y*X**3 + C[12]*X**2*Y**2 + C[13]*X**4 + C[14]*Y**4 \
        \
        + C[15]*X*Y**4 + C[16]*Y*X**4 + C[17]*X**2*Y**3 + C[18]*X**3*Y**2 + C[19]*X**5 + C[20]*Y**5 \
        \
        + C[21]*X**5*Y + C[22]*Y**5*X +  C[23]*X**4*Y**2 + C[24]*Y**4*X**2 + C[25]*X**3*Y**3 + C[26]*X**6 \
        \
        + C[27]*Y**6 + C[28]*X**6*Y + C[29]*X*Y**6 + C[30]*X**5*Y**2 + C[31]*X**2*Y**5 +C[32]*X**4*Y**3 + C[33]*X**3*Y**4 +\
        C[34]*X**7 + C[35]*Y**7 \
        \
        + C[36]*X**7*Y + C[37]*X*Y**7 + C[38]*X**6*Y**2 + C[39]*Y**6*X**2 + C[40]*X**5*Y**3 + C[41]*Y**5*X**3 + C[42]*X**4*Y**4 +\
        C[43]*X**8 + C[44]*Y**8\
        \
        + C[45]*X**8*Y + C[46]*X*Y**8 + C[47]*X**7*Y**2 + C[48]*Y**7*X**2 + C[49]*X**6*Y**3 + C[50]*Y**6*X**3 + C[51]*X**5*Y**4 +\
        C[52]*Y**5*X**4 + C[53]*X**9 + C[54]*Y**9\
        \
        + C[55]*X**9*Y + C[56]*X*Y**9 + C[57]*X**8*Y**2 + C[58]*Y**8*X**2 + C[59]*X**7*Y**3 + C[60]*Y**7*X**3 + C[61]*X**6*Y**4 +\
        C[62]*Y**6*X**4 + C[63]*X**5*Y**5 + C[64]*X**10 + C[65]*Y**10

    elif order == 11:
        A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1), data[:,:2]**2,\
                  \
                  data[:,0]*(data[:,1]**2),data[:,1]*(data[:,0]**2),data[:,:2]**3\
                  \
                  ,data[:,0]*data[:,1]**3,data[:,1]*data[:,0]**3,data[:,0]**2*data[:,1]**2,data[:,:2]**4\
                  \
                  ,data[:,0]*data[:,1]**4,data[:,1]*data[:,0]**4,data[:,0]**2*data[:,1]**3,data[:,1]**2*data[:,0]**3,data[:,:2]**5\
                  \
                  ,data[:,0]**5*data[:,1],data[:,1]**5*data[:,0], data[:,0]**4*data[:,1]**2, data[:,1]**4*data[:,0]**2,
                  data[:,0]**3*data[:,1]**3,data[:,:2]**6\
                  \
                  ,data[:,0]**6*data[:,1],data[:,1]**6*data[:,0],data[:,0]**5*data[:,1]**2,data[:,1]**5*data[:,0]**2\
                  ,data[:,0]**4*data[:,1]**3,data[:,1]**4*data[:,0]**3,data[:,:2]**7\
                  \
                  ,data[:,0]**7*data[:,1],data[:,1]**7*data[:,0], data[:,0]**6*data[:,1]**2, data[:,1]**6*data[:,0]**2,\
                  data[:,0]**5*data[:,1]**3, data[:,1]**5*data[:,0]**3, data[:,0]**4*data[:,1]**4, data[:,:2]**8,\
                 \
                 data[:,0]**8*data[:,1],data[:,1]**8*data[:,0], data[:,0]**7*data[:,1]**2, data[:,1]**7*data[:,0]**2,\
                  data[:,0]**6*data[:,1]**3, data[:,1]**6*data[:,0]**3, data[:,0]**5*data[:,1]**4, data[:,1]**5*data[:,0]**4,\
                  data[:,:2]**9\
                 \
                 ,data[:,0]**9*data[:,1],data[:,1]**9*data[:,0], data[:,0]**8*data[:,1]**2, data[:,1]**8*data[:,0]**2,\
                  data[:,0]**7*data[:,1]**3, data[:,1]**7*data[:,0]**3, data[:,0]**6*data[:,1]**4, data[:,1]**6*data[:,0]**4,\
                  data[:,0]**5*data[:,1]**5,data[:,:2]**10\
                 \
                 ,data[:,0]**10*data[:,1],data[:,1]**10*data[:,0], data[:,0]**9*data[:,1]**2, data[:,1]**9*data[:,0]**2,\
                  data[:,0]**8*data[:,1]**3, data[:,1]**8*data[:,0]**3, data[:,0]**7*data[:,1]**4, data[:,1]**7*data[:,0]**4,\
                  data[:,0]**6*data[:,1]**5,data[:,1]**6*data[:,0]**5, data[:,:2]**11]

        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])
        # evaluate it on a grid
        Z  = C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] \
        \
        + C[6]*X*Y*Y + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 \
        \
        + C[10]*X*Y**3 + C[11]*Y*X**3 + C[12]*X**2*Y**2 + C[13]*X**4 + C[14]*Y**4 \
        \
        + C[15]*X*Y**4 + C[16]*Y*X**4 + C[17]*X**2*Y**3 + C[18]*X**3*Y**2 + C[19]*X**5 + C[20]*Y**5 \
        \
        + C[21]*X**5*Y + C[22]*Y**5*X +  C[23]*X**4*Y**2 + C[24]*Y**4*X**2 + C[25]*X**3*Y**3 + C[26]*X**6 + C[27]*Y**6 \
        \
        + C[28]*X**6*Y + C[29]*X*Y**6 + C[30]*X**5*Y**2 + C[31]*X**2*Y**5 +C[32]*X**4*Y**3 + C[33]*X**3*Y**4 +\
        C[34]*X**7 + C[35]*Y**7 \
        \
        + C[36]*X**7*Y + C[37]*X*Y**7 + C[38]*X**6*Y**2 + C[39]*Y**6*X**2 + C[40]*X**5*Y**3 + C[41]*Y**5*X**3 + C[42]*X**4*Y**4 +\
        C[43]*X**8 + C[44]*Y**8\
        \
        + C[45]*X**8*Y + C[46]*X*Y**8 + C[47]*X**7*Y**2 + C[48]*Y**7*X**2 + C[49]*X**6*Y**3 + C[50]*Y**6*X**3 + C[51]*X**5*Y**4 +\
        C[52]*Y**5*X**4 + C[53]*X**9 + C[54]*Y**9\
        \
        + C[55]*X**9*Y + C[56]*X*Y**9 + C[57]*X**8*Y**2 + C[58]*Y**8*X**2 + C[59]*X**7*Y**3 + C[60]*Y**7*X**3 + C[61]*X**6*Y**4 +\
        C[62]*Y**6*X**4 + C[63]*X**5*Y**5 + C[64]*X**10 + C[65]*Y**10\
        \
        + C[66]*X**(10)*Y + C[67]*X*Y**(10) + C[68]*X**9*Y**2 + C[69]*Y**9*X**2 + C[70]*X**8*Y**3 + C[71]*Y**8*X**3 + C[72]*X**7*Y**4 \
        + C[73]*Y**7*X**4 + C[74]*X**6*Y**5 + C[75]*X**5*Y**6 + C[76]*X**(11) + C[77]*Y**(11)
         
    elif order == 12:
        A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1), data[:,:2]**2,\
                  \
                  data[:,0]*(data[:,1]**2),data[:,1]*(data[:,0]**2),data[:,:2]**3\
                  \
                  ,data[:,0]*data[:,1]**3,data[:,1]*data[:,0]**3,data[:,0]**2*data[:,1]**2,data[:,:2]**4\
                  \
                  ,data[:,0]*data[:,1]**4,data[:,1]*data[:,0]**4,data[:,0]**2*data[:,1]**3,data[:,1]**2*data[:,0]**3,data[:,:2]**5\
                  \
                  ,data[:,0]**5*data[:,1],data[:,1]**5*data[:,0], data[:,0]**4*data[:,1]**2, data[:,1]**4*data[:,0]**2,
                  data[:,0]**3*data[:,1]**3,data[:,:2]**6\
                  \
                  ,data[:,0]**6*data[:,1],data[:,1]**6*data[:,0],data[:,0]**5*data[:,1]**2,data[:,1]**5*data[:,0]**2\
                  ,data[:,0]**4*data[:,1]**3,data[:,1]**4*data[:,0]**3,data[:,:2]**7\
                  \
                  ,data[:,0]**7*data[:,1],data[:,1]**7*data[:,0], data[:,0]**6*data[:,1]**2, data[:,1]**6*data[:,0]**2,\
                  data[:,0]**5*data[:,1]**3, data[:,1]**5*data[:,0]**3, data[:,0]**4*data[:,1]**4, data[:,:2]**8,\
                 \
                 data[:,0]**8*data[:,1],data[:,1]**8*data[:,0], data[:,0]**7*data[:,1]**2, data[:,1]**7*data[:,0]**2,\
                  data[:,0]**6*data[:,1]**3, data[:,1]**6*data[:,0]**3, data[:,0]**5*data[:,1]**4, data[:,1]**5*data[:,0]**4,\
                  data[:,:2]**9\
                 \
                 ,data[:,0]**9*data[:,1],data[:,1]**9*data[:,0], data[:,0]**8*data[:,1]**2, data[:,1]**8*data[:,0]**2,\
                  data[:,0]**7*data[:,1]**3, data[:,1]**7*data[:,0]**3, data[:,0]**6*data[:,1]**4, data[:,1]**6*data[:,0]**4,\
                  data[:,0]**5*data[:,1]**5,data[:,:2]**10\
                 \
                 ,data[:,0]**10*data[:,1],data[:,1]**10*data[:,0], data[:,0]**9*data[:,1]**2, data[:,1]**9*data[:,0]**2,\
                  data[:,0]**8*data[:,1]**3, data[:,1]**8*data[:,0]**3, data[:,0]**7*data[:,1]**4, data[:,1]**7*data[:,0]**4,\
                  data[:,0]**6*data[:,1]**5,data[:,1]**6*data[:,0]**5, data[:,:2]**11\
                  \
                 ,data[:,0]**11*data[:,1],data[:,1]**11*data[:,0], data[:,0]**10*data[:,1]**2, data[:,1]**10*data[:,0]**2,\
                  data[:,0]**9*data[:,1]**3, data[:,1]**9*data[:,0]**3, data[:,0]**8*data[:,1]**4, data[:,1]**8*data[:,0]**4,\
                  data[:,0]**7*data[:,1]**5,data[:,1]**7*data[:,0]**5, data[:,0]**6*data[:,1]**6, data[:,:2]**12]

        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])
        # evaluate it on a grid
        Z  = C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] \
        \
        + C[6]*X*Y*Y + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 \
        \
        + C[10]*X*Y**3 + C[11]*Y*X**3 + C[12]*X**2*Y**2 + C[13]*X**4 + C[14]*Y**4 \
        \
        + C[15]*X*Y**4 + C[16]*Y*X**4 + C[17]*X**2*Y**3 + C[18]*X**3*Y**2 + C[19]*X**5 + C[20]*Y**5 \
        \
        + C[21]*X**5*Y + C[22]*Y**5*X +  C[23]*X**4*Y**2 + C[24]*Y**4*X**2 + C[25]*X**3*Y**3 + C[26]*X**6 + C[27]*Y**6 \
        \
        + C[28]*X**6*Y + C[29]*X*Y**6 + C[30]*X**5*Y**2 + C[31]*X**2*Y**5 +C[32]*X**4*Y**3 + C[33]*X**3*Y**4 +\
        C[34]*X**7 + C[35]*Y**7 \
        \
        + C[36]*X**7*Y + C[37]*X*Y**7 + C[38]*X**6*Y**2 + C[39]*Y**6*X**2 + C[40]*X**5*Y**3 + C[41]*Y**5*X**3 + C[42]*X**4*Y**4 +\
        C[43]*X**8 + C[44]*Y**8\
        \
        + C[45]*X**8*Y + C[46]*X*Y**8 + C[47]*X**7*Y**2 + C[48]*Y**7*X**2 + C[49]*X**6*Y**3 + C[50]*Y**6*X**3 + C[51]*X**5*Y**4 +\
        C[52]*Y**5*X**4 + C[53]*X**9 + C[54]*Y**9\
        \
        + C[55]*X**9*Y + C[56]*X*Y**9 + C[57]*X**8*Y**2 + C[58]*Y**8*X**2 + C[59]*X**7*Y**3 + C[60]*Y**7*X**3 + C[61]*X**6*Y**4 +\
        C[62]*Y**6*X**4 + C[63]*X**5*Y**5 + C[64]*X**10 + C[65]*Y**10\
        \
        + C[66]*X**(10)*Y + C[67]*X*Y**(10) + C[68]*X**9*Y**2 + C[69]*Y**9*X**2 + C[70]*X**8*Y**3 + C[71]*Y**8*X**3 + C[72]*X**7*Y**4 \
        + C[73]*Y**7*X**4 + C[74]*X**6*Y**5 + C[75]*X**5*Y**6 + C[76]*X**(11) + C[77]*Y**(11)\
        \
        + C[78]*X**11*Y + C[79]*X*Y**11 + C[80]*X**10*Y**2 + C[81]*Y**10*X**2 + C[82]*X**9*Y**3 + C[83]*Y**9*X**3 + C[84]*X**8*Y**4 \
        + C[85]*Y**8*X**4 + C[86]*X**7*Y**5 + C[87]*X**5*Y**7 + C[88]*X**6*Y**6 + C[89]*X**12 + C[90]*Y**12 
 
    for i in range(plot):
        if plot==1:
            #plot points and fitted surface
            fig = plt.figure()
            ax = fig.gca(projection='3d')
            ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.9, cmap=cm.magma)
            ax.scatter(data[:,0], data[:,1], data[:,2], c='deeppink', s=8)
            plt.xlabel('Q1')
            plt.ylabel('Q2')
            ax.set_zlabel('mu')
            ax.axis('equal')
            ax.axis('tight')
            axes = plt.gca()
            plt.show()
            #plt.savefig('plot'+str(T)+str(order),dpi=150)
    
    return C

In [ ]:
def second_derivative(T,S):
    """
    Calculates the second derivatives of fits from second to 11th order based on the fit from "fit"
    given the desired number of subtracted data points, S
    """
    X = sympy.Symbol('X')
    Y = sympy.Symbol('Y')
    D_list=[]

    C = fit(T,2,S)
    function= C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] \
         
    function_dx=function.diff(X)
    function_dxdy=function_dx.diff(Y)
    D_list.append(function_dxdy.subs([(X,0), (Y,0)]))

    C = fit(T,3,S)
    function= C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] + C[6]*X*Y*Y \
    + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 
    
    function_dx=function.diff(X)
    function_dxdy=function_dx.diff(Y)
    D_list.append(function_dxdy.subs([(X,0), (Y,0)]))

    C = fit(T,4,S)
    function= C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] + C[6]*X*Y*Y \
    + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3+ C[10]*X*Y**3 + C[11]*Y*X**3 + C[12]*X**2*Y**2 \
    + C[13]*X**4 + C[14]*Y**4\
    
    function_dx=function.diff(X)
    function_dxdy=function_dx.diff(Y)
    D_list.append(function_dxdy.subs([(X,0), (Y,0)]))

    C = fit(T,5,S)
    function=  C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] + C[6]*X*Y*Y\
    + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 + C[10]*X*Y**3 + C[11]*Y*X**3 + C[12]*X**2*Y**2 + \
    C[13]*X**4 + C[14]*Y**4+ C[15]*X*Y**4 + C[16]*Y*X**4 + C[17]*X**2*Y**3 + \
    C[18]*X**3*Y**2 + C[19]*X**5 + C[20]*Y**5 \
    
    function_dx=function.diff(X)
    function_dxdy=function_dx.diff(Y)
    D_list.append(function_dxdy.subs([(X,0), (Y,0)]))


    C = fit(T,6,S)
    function= C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] +\
    C[6]*X*Y*Y + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 + C[10]*X*Y**3 + C[11]*Y*X**3\
    + C[12]*X**2*Y**2 + C[13]*X**4 + C[14]*Y**4 + C[15]*X*Y**4 + C[16]*Y*X**4 \
    +C[17]*X**2*Y**3+ C[18]*X**3*Y**2 + C[19]*X**5 + C[20]*Y**5 + C[21]*X**5*Y \
    + C[22]*Y**5*X +  C[23]*X**4*Y**2 + C[24]*Y**4*X**2 + C[25]*X**3*Y**3 +\
    C[26]*X**6 + C[27]*Y**6 \
    
    function_dx=function.diff(X)
    function_dxdy=function_dx.diff(Y)
    D_list.append(function_dxdy.subs([(X,0), (Y,0)]))

    C = fit(T,7,S)
    function= C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] +\
    C[6]*X*Y*Y + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 + C[10]*X*Y**3 + C[11]*Y*X**3\
    + C[12]*X**2*Y**2 + C[13]*X**4 + C[14]*Y**4 + C[15]*X*Y**4 + C[16]*Y*X**4 +\
    C[17]*X**2*Y**3 + C[18]*X**3*Y**2 + C[19]*X**5 + C[20]*Y**5 + C[21]*X**5*Y +\
    C[22]*Y**5*X +  C[23]*X**4*Y**2 +C[24]*Y**4*X**2 + C[25]*X**3*Y**3 +\
    C[26]*X**6 + C[27]*Y**6 + C[28]*X**6*Y + C[29]*X*Y**6 + C[30]*X**5*Y**2 + \
    C[31]*X**2*Y**5 +C[32]*X**4*Y**3 + C[33]*X**3*Y**4 + C[34]*X**7 + C[35]*Y**7 \
    
    function_dx=function.diff(X)
    function_dxdy=function_dx.diff(Y)
    D_list.append(function_dxdy.subs([(X,0), (Y,0)]))

    C = fit(T,8,S)
    function= C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] + \
    C[6]*X*Y*Y + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 + C[10]*X*Y**3 + C[11]*Y*X**3\
    + C[12]*X**2*Y**2 + C[13]*X**4 + C[14]*Y**4 + C[15]*X*Y**4 + C[16]*Y*X**4 +\
    C[17]*X**2*Y**3 + C[18]*X**3*Y**2 + C[19]*X**5 + C[20]*Y**5 + C[21]*X**5*Y + \
    C[22]*Y**5*X +  C[23]*X**4*Y**2 +C[24]*Y**4*X**2 + C[25]*X**3*Y**3 +\
    C[26]*X**6 + C[27]*Y**6 + C[28]*X**6*Y + C[29]*X*Y**6 + C[30]*X**5*Y**2 + \
    C[31]*X**2*Y**5 +C[32]*X**4*Y**3 + C[33]*X**3*Y**4 +C[34]*X**7 + C[35]*Y**7\
    + C[36]*X**7*Y + C[37]*X*Y**7 + C[38]*X**6*Y**2 + C[39]*Y**6*X**2 +\
    C[40]*X**5*Y**3 + C[41]*Y**5*X**3 + C[42]*X**4*Y**4 + C[43]*X**8 + C[44]*Y**8\

    function_dx=function.diff(X)
    function_dxdy=function_dx.diff(Y)
    D_list.append(function_dxdy.subs([(X,0), (Y,0)]))

    C = fit(T,9,S)
    function= C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] + C[6]*X*Y*Y\
    + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 + C[10]*X*Y**3 + C[11]*Y*X**3 + C[12]*X**2*Y**2\
    + C[13]*X**4 + C[14]*Y**4 + C[15]*X*Y**4 + C[16]*Y*X**4 + C[17]*X**2*Y**3 +\
    C[18]*X**3*Y**2 + C[19]*X**5 + C[20]*Y**5 + C[21]*X**5*Y + C[22]*Y**5*X +  C[23]*X**4*Y**2 \
    + C[24]*Y**4*X**2 + C[25]*X**3*Y**3 + C[26]*X**6 + C[27]*Y**6 + C[28]*X**6*Y + \
    C[29]*X*Y**6 + C[30]*X**5*Y**2 +C[31]*X**2*Y**5 +C[32]*X**4*Y**3 + C[33]*X**3*Y**4\
    + C[34]*X**7 + C[35]*Y**7 + C[36]*X**7*Y + C[37]*X*Y**7 +C[38]*X**6*Y**2 + \
    C[39]*Y**6*X**2 + C[40]*X**5*Y**3 + C[41]*Y**5*X**3 + C[42]*X**4*Y**4 + C[43]*X**8 + C[44]*Y**8\
    + C[45]*X**8*Y + C[46]*X*Y**8 + C[47]*X**7*Y**2 + C[48]*Y**7*X**2 + C[49]*X**6*Y**3 \
    + C[50]*Y**6*X**3 + C[51]*X**5*Y**4 +C[52]*Y**5*X**4 + C[53]*X**9 + C[54]*Y**9\


    function_dx=function.diff(X)
    function_dxdy=function_dx.diff(Y)
    D_list.append(function_dxdy.subs([(X,0), (Y,0)]))

    C = fit(T,10,S)
    function= C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] + C[6]*X*Y*Y\
    + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 + C[10]*X*Y**3 + C[11]*Y*X**3 + C[12]*X**2*Y**2\
    + C[13]*X**4 + C[14]*Y**4 + C[15]*X*Y**4 + C[16]*Y*X**4 + C[17]*X**2*Y**3\
    + C[18]*X**3*Y**2 + C[19]*X**5 + C[20]*Y**5 + C[21]*X**5*Y + C[22]*Y**5*X +\
    C[23]*X**4*Y**2 + C[24]*Y**4*X**2 + C[25]*X**3*Y**3 + C[26]*X**6 + C[27]*Y**6\
    + C[28]*X**6*Y + C[29]*X*Y**6 + C[30]*X**5*Y**2 + C[31]*X**2*Y**5 +C[32]*X**4*Y**3\
    + C[33]*X**3*Y**4 +C[34]*X**7 + C[35]*Y**7 + C[36]*X**7*Y + C[37]*X*Y**7 + C[38]*X**6*Y**2 + \
    C[39]*Y**6*X**2 + C[40]*X**5*Y**3 + C[41]*Y**5*X**3 + C[42]*X**4*Y**4 + C[43]*X**8 +\
    C[44]*Y**8+ C[45]*X**8*Y +C[46]*X*Y**8 + C[47]*X**7*Y**2 + C[48]*Y**7*X**2 + \
    C[49]*X**6*Y**3 + C[50]*Y**6*X**3 + C[51]*X**5*Y**4 +C[52]*Y**5*X**4 +C[53]*X**9 + C[54]*Y**9\
    + C[55]*X**9*Y + C[56]*X*Y**9 + C[57]*X**8*Y**2 + C[58]*Y**8*X**2 + C[59]*X**7*Y**3 + \
    C[60]*Y**7*X**3 + C[61]*X**6*Y**4 + C[62]*Y**6*X**4 + C[63]*X**5*Y**5 + C[64]*X**10 + C[65]*Y**10\

    function_dx=function.diff(X)
    function_dxdy=function_dx.diff(Y)
    D_list.append(function_dxdy.subs([(X,0), (Y,0)]))

    C = fit(T,11,S)
    function= C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0] + C[6]*X*Y*Y\
    + C[7]*Y*X*X + C[8]*X**3 +C[9]*Y**3 + C[10]*X*Y**3 + C[11]*Y*X**3 + C[12]*X**2*Y**2\
    + C[13]*X**4 + C[14]*Y**4 + C[15]*X*Y**4 + C[16]*Y*X**4 + C[17]*X**2*Y**3 + \
    C[18]*X**3*Y**2 + C[19]*X**5 + C[20]*Y**5 + C[21]*X**5*Y + C[22]*Y**5*X +  C[23]*X**4*Y**2\
    + C[24]*Y**4*X**2 + C[25]*X**3*Y**3 + C[26]*X**6 + C[27]*Y**6 + C[28]*X**6*Y + \
    C[29]*X*Y**6 + C[30]*X**5*Y**2 + C[31]*X**2*Y**5 +C[32]*X**4*Y**3 + C[33]*X**3*Y**4\
    +C[34]*X**7 + C[35]*Y**7 + C[36]*X**7*Y + C[37]*X*Y**7 +C[38]*X**6*Y**2 + C[39]*Y**6*X**2\
    + C[40]*X**5*Y**3 + C[41]*Y**5*X**3 + C[42]*X**4*Y**4 + C[43]*X**8 + C[44]*Y**8\
    + C[45]*X**8*Y + C[46]*X*Y**8 + C[47]*X**7*Y**2 + C[48]*Y**7*X**2 + C[49]*X**6*Y**3\
    + C[50]*Y**6*X**3 + C[51]*X**5*Y**4 + C[52]*Y**5*X**4 + C[53]*X**9 + C[54]*Y**9 + C[55]*X**9*Y \
    + C[56]*X*Y**9 + C[57]*X**8*Y**2 + C[58]*Y**8*X**2 + C[59]*X**7*Y**3 + C[60]*Y**7*X**3\
    + C[61]*X**6*Y**4 + C[62]*Y**6*X**4 + C[63]*X**5*Y**5 + C[64]*X**10 + C[65]*Y**10+ \
    C[66]*X**(10)*Y + C[67]*X*Y**(10) + C[68]*X**9*Y**2 + C[69]*Y**9*X**2 + C[70]*X**8*Y**3 \
    + C[71]*Y**8*X**3 + C[72]*X**7*Y**4+ C[73]*Y**7*X**4 + C[74]*X**6*Y**5 +\
    C[75]*X**5*Y**6 + C[76]*X**(11) + C[77]*Y**(11)\

    function_dx=function.diff(X)
    function_dxdy=function_dx.diff(Y)
    D_list.append(function_dxdy.subs([(X,0), (Y,0)]))

    return(D_list)

In [ ]:
def hej(stepsize, red_mass1, red_mass2):
    """
    Calculates the conversion factor that converts the axes since 1 on the axes actually correspond to stepsize.  
    """
    conversion=stepsize**2
    return conversion

In [ ]:
def mu_len():
    """
    Fits the transition dipole moment 20 times for each order of polynomial with 6% of the data points removed before
    choosing the best fit. Prints the obtained x, y and z derivatives as well as the average deviation in % 
    for the best fit. 
    """
    
    dx_list=second_derivative(1,0)
    dy_list=second_derivative(2,0)
    dz_list=second_derivative(3,0)
    tot_difference_list=np.zeros(len(dy_list))
 
    points=22
    
    for i in range(20):
        dify_list=second_derivative(2,points)
        difference_list=[]
        for j in range(len(dy_list)):
            x=0
            x=((dy_list[j]-dify_list[j])/dy_list[j])*100
            difference_list.append(abs(x))
        difference_list=np.array(difference_list)
        tot_difference_list=tot_difference_list +difference_list
    
    b=np.argmin(tot_difference_list)
    
    deriv_y=dy_list[b]
    mini=min(tot_difference_list)
    print(deriv_y)
    print('dify='+str(mini/20))
          
    tot_difference_list=np.zeros(len(dz_list))
    
    
    for i in range(20):
        difz_list=second_derivative(3,points)
        difference_list=[]
        for j in range(len(dz_list)):
            x=0
            x=((dz_list[j]-difz_list[j])/dz_list[j])*100
            difference_list.append(abs(x))
        difference_list=np.array(difference_list)
        tot_difference_list=tot_difference_list +difference_list
    
    b=np.argmin(tot_difference_list)
    
    deriv_z=dz_list[b]
    print(deriv_z)
    minimum=min(tot_difference_list)
    print('difz='+str(minimum/20))
    
    tot_difference_list=np.zeros(len(dx_list))
    for i in range(20):
        difx_list=second_derivative(1,points)
        difference_list=[]
        for j in range(len(dz_list)):
            x=0
            x=((dx_list[j]-difx_list[j])/dx_list[j])*100
            difference_list.append(abs(x))
        difference_list=np.array(difference_list)
        tot_difference_list=tot_difference_list +difference_list
    
    b=np.argmin(tot_difference_list)
    deriv_x=dx_list[b]
    minimum=min(tot_difference_list)
    print(deriv_x)
    
    print('difx='+str(minimum/20))
    
    mu=np.zeros(3)
    mu[0]=deriv_x
    mu[1]=deriv_y
    mu[2]=deriv_z
    mu_l=np.sqrt(mu@mu)
    return mu_l

In [ ]:
def osc(m1,m2,nu1,nu2,ex_freq, stepsize):
    """
    Calculates the Oscillator strength from the derivatives of the x, y and z components. 
    """
    h=6.62608*10**(-34) #Placnk's constant
    me=9.10938*10**(-31) #mass of electron
    a_m=10**(-10) #Ångstrom til meter conversion factor
    
    conversion=hej(stepsize, m1, m2)
    
    Q1_squared= h/(m1*8*np.pi**2*nu1)
    Q2_squared= h/(m2*8*np.pi**2*nu2)

    f= (8*np.pi**2*me)/(3*h)*ex_freq*((mu_len()/a_m)*conversion**(-1))**2*Q1_squared*Q2_squared
    
    return f
    

In [ ]:
"""
Runs osc(mu1,mu1,v1,v2,ex_freq,stepsize) to give the oscillator strength. The number of vibrations must be given specifically as
vib1 and vib2
"""

plot=0
file='methanol_wb_TZ.log' #a log file containing the ground state vibrational frequencies
file2= 'td.log' #a log file containing the td-DFT calculation of the ground state structure

freq_list, red_list= extract_freq(file)
ex_freq=oscillator(file2)

stepsize=0.001 #The stepsize
vib1=5 #vibrational mode 1
vib2=12 #vibrational mode 2

osc(red_list[vib1-1], red_list[vib2-1],freq_list[vib1-1], freq_list[vib2-1],ex_freq,stepsize)